In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [1]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
import os
import keras

dataset_path = os.listdir('/kaggle/input/shop-dataset/Shop DataSet')

label_types = os.listdir('/kaggle/input/shop-dataset/Shop DataSet')
print (label_types) 

['non shop lifters', 'shop lifters']


In [2]:
rooms = []

for item in dataset_path:
 # Get all the file names
 all_rooms = os.listdir('/kaggle/input/shop-dataset/Shop DataSet' + '/' +item)

 # Add them to the list
 for room in all_rooms:
    rooms.append((item, str('/kaggle/input/shop-dataset/Shop DataSet' + '/' +item) + '/' + room))
    
# Build a dataframe        
dataSet_df = pd.DataFrame(data=rooms, columns=['tag', 'video_name'])
print(dataSet_df.head())
print(dataSet_df.tail())

                tag                                         video_name
0  non shop lifters  /kaggle/input/shop-dataset/Shop DataSet/non sh...
1  non shop lifters  /kaggle/input/shop-dataset/Shop DataSet/non sh...
2  non shop lifters  /kaggle/input/shop-dataset/Shop DataSet/non sh...
3  non shop lifters  /kaggle/input/shop-dataset/Shop DataSet/non sh...
4  non shop lifters  /kaggle/input/shop-dataset/Shop DataSet/non sh...
              tag                                         video_name
850  shop lifters  /kaggle/input/shop-dataset/Shop DataSet/shop l...
851  shop lifters  /kaggle/input/shop-dataset/Shop DataSet/shop l...
852  shop lifters  /kaggle/input/shop-dataset/Shop DataSet/shop l...
853  shop lifters  /kaggle/input/shop-dataset/Shop DataSet/shop l...
854  shop lifters  /kaggle/input/shop-dataset/Shop DataSet/shop l...


In [3]:
df = dataSet_df.loc[:,['video_name','tag']]
df
df.to_csv('dataSet.csv')

In [4]:
dataSet_df = pd.read_csv("dataSet.csv")
print(f"Total videos in dataSet: {len(dataSet_df)}")
dataSet_df.sample(10)

Total videos in dataSet: 855


,Unnamed: 0,video_name,tag
761,761,/kaggle/input/shop-dataset/Shop DataSet/shop l...,shop lifters
197,197,/kaggle/input/shop-dataset/Shop DataSet/non sh...,non shop lifters
430,430,/kaggle/input/shop-dataset/Shop DataSet/non sh...,non shop lifters
651,651,/kaggle/input/shop-dataset/Shop DataSet/shop l...,shop lifters
71,71,/kaggle/input/shop-dataset/Shop DataSet/non sh...,non shop lifters
723,723,/kaggle/input/shop-dataset/Shop DataSet/shop l...,shop lifters
294,294,/kaggle/input/shop-dataset/Shop DataSet/non sh...,non shop lifters
368,368,/kaggle/input/shop-dataset/Shop DataSet/non sh...,non shop lifters
385,385,/kaggle/input/shop-dataset/Shop DataSet/non sh...,non shop lifters
712,712,/kaggle/input/shop-dataset/Shop DataSet/shop l...,shop lifters


In [5]:
df = dataSet_df.copy()

In [6]:
import cv2
import numpy as np

def extract_and_sample_frames(video_path, num_frames=16):
    cap = cv2.VideoCapture(video_path)
    frames = []
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

    # Calculate frame sampling interval
    sample_interval = max(1, total_frames // num_frames)

    count = 0
    while len(frames) < num_frames and cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        # Sample frames at regular intervals
        if count % sample_interval == 0:
            frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            frames.append(frame_rgb)

        count += 1

    # If fewer frames are extracted, pad with repeated frames
    while len(frames) < num_frames:
        frames.append(frames[-1])

    cap.release()
    return frames


In [7]:
from transformers import VideoMAEFeatureExtractor, VideoMAEForVideoClassification
import torch

# Load pre-trained video transformer
feature_extractor = VideoMAEFeatureExtractor.from_pretrained("MCG-NJU/videomae-base")
model = VideoMAEForVideoClassification.from_pretrained("MCG-NJU/videomae-base")

# Preprocess frames
def preprocess_video_frames(video_path):
    frames = extract_and_sample_frames(video_path, num_frames=16)
    inputs = feature_extractor(frames, return_tensors="pt")
    return inputs


preprocessor_config.json:   0%|          | 0.00/271 [00:00<?, ?B/s]

/opt/conda/lib/python3.10/site-packages/transformers/models/videomae/feature_extraction_videomae.py:28: FutureWarning: The class VideoMAEFeatureExtractor is deprecated and will be removed in version 5 of Transformers. Please use VideoMAEImageProcessor instead.
  warnings.warn(


config.json:   0%|          | 0.00/725 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/377M [00:00<?, ?B/s]

Some weights of VideoMAEForVideoClassification were not initialized from the model checkpoint at MCG-NJU/videomae-base and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [9]:
from tqdm import tqdm  # Import tqdm for progress bar
import torch

video_paths = df['video_name'].tolist()  # Replace with your video path column name
labels = df['tag'].tolist() 

# Define a mapping from string labels to numeric labels
label_mapping = {
    'shop lifters': 1,
    'non shop lifters': 0
}

# Preprocess all videos
inputs_list = []
labels_list = []

# Use tqdm to wrap the zip iterator
for video_path, label in tqdm(zip(video_paths, labels), total=len(video_paths), desc="Processing videos"):
    inputs = preprocess_video_frames(video_path)
    inputs_list.append(inputs['pixel_values'])  # Append preprocessed video

    # Convert label to numeric form and append to the list
    numeric_label = label_mapping[label]
    labels_list.append(numeric_label)

# Stack inputs and labels
inputs_tensor = torch.cat(inputs_list)
labels_tensor = torch.tensor(labels_list)

print(f"Inputs tensor shape: {inputs_tensor.shape}")
print(f"Labels tensor shape: {labels_tensor.shape}")


Processing videos:   0%|          | 0/855 [00:00<?, ?it/s]/opt/conda/lib/python3.10/site-packages/transformers/feature_extraction_utils.py:142: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at /usr/local/src/pytorch/torch/csrc/utils/tensor_new.cpp:278.)
  return torch.tensor(value)
Processing videos: 100%|██████████| 855/855 [14:00<00:00,  1.02it/s]


Inputs tensor shape: torch.Size([855, 16, 3, 224, 224])
Labels tensor shape: torch.Size([855])


In [11]:
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader, TensorDataset, random_split

# First, ensure tensors have the correct shapes
print(f"Inputs tensor shape: {inputs_tensor.shape}")
print(f"Labels tensor shape: {labels_tensor.shape}")

# Create a dataset using TensorDataset
dataset = TensorDataset(inputs_tensor, labels_tensor)

# Define the split proportions
train_ratio = 0.75
valid_ratio = 0.10
test_ratio = 0.15

# Calculate the lengths of each split
dataset_size = len(dataset)
train_size = int(train_ratio * dataset_size)
valid_size = int(valid_ratio * dataset_size)
test_size = dataset_size - train_size - valid_size

# Split the dataset into training, validation, and test sets
train_dataset, valid_dataset, test_dataset = random_split(dataset, [train_size, valid_size, test_size])

# Create dataloaders for each split
train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True)
valid_loader = DataLoader(valid_dataset, batch_size=8, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=8, shuffle=False)

print(f"Train size: {train_size}, Valid size: {valid_size}, Test size: {test_size}")


Inputs tensor shape: torch.Size([855, 16, 3, 224, 224])
Labels tensor shape: torch.Size([855])
Train size: 641, Valid size: 85, Test size: 129


In [12]:
import torch
from torch.utils.data import DataLoader, TensorDataset
from tqdm import tqdm

# Check if a GPU is available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Move model to the GPU
model = model.to(device)

# Create a dataset and dataloader
train_dataset = TensorDataset(inputs_tensor, labels_tensor)  # Assuming you've already split your data
train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True)

# Assuming you have a validation dataset
valid_loader = DataLoader(valid_dataset, batch_size=8, shuffle=False)

# Define optimizer and loss function
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)
loss_fn = torch.nn.CrossEntropyLoss()

# Function to calculate accuracy
def calculate_accuracy(preds, labels):
    _, predicted_classes = torch.max(preds, 1)
    correct_predictions = (predicted_classes == labels).sum().item()
    return correct_predictions / labels.size(0)

# Training loop
num_epochs = 5
for epoch in range(num_epochs):
    model.train()
    total_train_loss = 0
    total_train_acc = 0

    # Training
    for batch in tqdm(train_loader, desc=f"Training Epoch {epoch+1}/{num_epochs}"):
        inputs, labels = batch
        
        # Move inputs and labels to the GPU
        inputs = inputs.to(device)
        labels = labels.to(device)
        
        # Forward pass
        outputs = model(pixel_values=inputs)
        loss = loss_fn(outputs.logits, labels)

        # Backward pass and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # Accumulate loss and accuracy
        total_train_loss += loss.item()
        total_train_acc += calculate_accuracy(outputs.logits, labels)
    
    # Calculate average loss and accuracy for the epoch
    avg_train_loss = total_train_loss / len(train_loader)
    avg_train_acc = total_train_acc / len(train_loader)

    # Validation
    model.eval()
    total_valid_loss = 0
    total_valid_acc = 0
    with torch.no_grad():
        for batch in tqdm(valid_loader, desc=f"Validation Epoch {epoch+1}/{num_epochs}"):
            inputs, labels = batch
            
            # Move inputs and labels to the GPU
            inputs = inputs.to(device)
            labels = labels.to(device)
            
            # Forward pass
            outputs = model(pixel_values=inputs)
            loss = loss_fn(outputs.logits, labels)

            # Accumulate validation loss and accuracy
            total_valid_loss += loss.item()
            total_valid_acc += calculate_accuracy(outputs.logits, labels)

    # Calculate average validation loss and accuracy for the epoch
    avg_valid_loss = total_valid_loss / len(valid_loader)
    avg_valid_acc = total_valid_acc / len(valid_loader)

    # Print the results for the current epoch
    print(f"Epoch {epoch+1}/{num_epochs}")
    print(f"Training Loss: {avg_train_loss:.4f}, Training Accuracy: {avg_train_acc:.4f}")
    print(f"Validation Loss: {avg_valid_loss:.4f}, Validation Accuracy: {avg_valid_acc:.4f}")


Using device: cuda


Validation Epoch 1/5: 100%|██████████| 11/11 [00:05<00:00,  1.97it/s]


Epoch 1/5
Training Loss: 0.2805, Training Accuracy: 0.8563
Validation Loss: 0.0732, Validation Accuracy: 0.9318


Validation Epoch 2/5: 100%|██████████| 11/11 [00:05<00:00,  1.97it/s]


Epoch 2/5
Training Loss: 0.0590, Training Accuracy: 0.9755
Validation Loss: 0.0063, Validation Accuracy: 1.0000


...

Epoch 3/5
Training Loss: 0.1061, Training Accuracy: 0.9673
Validation Loss: 0.0121, Validation Accuracy: 1.0000


Validation Epoch 4/5: 100%|██████████| 11/11 [00:05<00:00,  1.97it/s]


Epoch 4/5
Training Loss: 0.0168, Training Accuracy: 0.9942
Validation Loss: 0.0016, Validation Accuracy: 1.0000


Validation Epoch 5/5: 100%|██████████| 11/11 [00:05<00:00,  1.97it/s]

Epoch 5/5
Training Loss: 0.0011, Training Accuracy: 1.0000
Validation Loss: 0.0003, Validation Accuracy: 1.0000


In [14]:
import torch
from tqdm import tqdm

# Define a mapping from the model's output to labels
label_mapping = {1: 'Shoplifter', 0: 'Non-Shoplifter'}

# Preprocessing function (same as used in training)
def preprocess_external_video(video_path):
    # Assuming you have a function 'extract_and_sample_frames' to extract video frames
    frames = extract_and_sample_frames(video_path, num_frames=16)  # Adjust the number of frames as needed
    inputs = feature_extractor(frames, return_tensors="pt")
    return inputs['pixel_values']

# Prediction function
def predict_shoplifter(video_path, model, device):
    # Preprocess the external video
    pixel_values = preprocess_external_video(video_path)

    # Move the input to the device (GPU/CPU)
    pixel_values = pixel_values.to(device)

    # Set model to evaluation mode
    model.eval()

    # Run the model with no gradient calculation (for inference)
    with torch.no_grad():
        # Make prediction
        outputs = model(pixel_values=pixel_values)
        logits = outputs.logits

    # Get the predicted class (0: Non-Shoplifter, 1: Shoplifter)
    predicted_class = torch.argmax(logits, dim=1).item()

    # Map the predicted class to the actual label
    prediction_label = label_mapping[predicted_class]

    return prediction_label

# Example usage
external_video_path = "/kaggle/input/shop-dataset/Shop DataSet/shop lifters/shop_lifter_0.mp4"
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Make a prediction for an external video
prediction = predict_shoplifter(external_video_path, model, device)
print(f"The video is predicted to show: {prediction}")


The video is predicted to show: Shoplifter


In [16]:
from sklearn.metrics import confusion_matrix, classification_report
import torch
import numpy as np

# Function to evaluate the model on validation/test data and collect predictions and true labels
def evaluate_model(dataloader, model, device):
    model.eval()  # Set the model to evaluation mode
    all_preds = []
    all_labels = []

    with torch.no_grad():  # Disable gradient calculation for evaluation
        for batch in dataloader:
            inputs, labels = batch

            # Move inputs and labels to the GPU/CPU
            inputs = inputs.to(device)
            labels = labels.to(device)

            # Forward pass to get model predictions
            outputs = model(pixel_values=inputs)
            logits = outputs.logits

            # Get predicted class labels
            preds = torch.argmax(logits, dim=1)

            # Store the predictions and true labels
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

    return np.array(all_preds), np.array(all_labels)

# Function to print confusion matrix and classification report
def print_evaluation_metrics(dataloader, model, device):
    # Get predictions and true labels
    y_pred, y_true = evaluate_model(dataloader, model, device)

    # Print confusion matrix
    print("Confusion Matrix:")
    print(confusion_matrix(y_true, y_pred))

    # Print classification report (precision, recall, F1-score)
    print("\nClassification Report:")
    print(classification_report(y_true, y_pred, target_names=['Non-Shoplifter', 'Shoplifter']))

# Example usage
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Assuming you have a DataLoader for your validation or test data
print_evaluation_metrics(test_loader, model, device)


Confusion Matrix:
[[80  0]
 [ 0 49]]

Classification Report:
                precision    recall  f1-score   support

Non-Shoplifter       1.00      1.00      1.00        80
    Shoplifter       1.00      1.00      1.00        49

      accuracy                           1.00       129
     macro avg       1.00      1.00      1.00       129
  weighted avg       1.00      1.00      1.00       129

